# Lesson I5: Testing & Documentation

**Duration**: 75-90 minutes  
**Stage**: Intermediate (Building Skills)  
**Prerequisites**: All previous Intermediate lessons (I1-I4)

---

## 📋 What You'll Learn

This lesson teaches you how to write tests and documentation in Rust. You'll learn to use Rust's built-in testing framework, write effective unit and integration tests, and create documentation that doubles as executable tests.

**Why this matters**: Testing is essential for reliable software. Rust's testing tools are integrated into the language and Cargo, making it easy to write and run tests. Documentation tests ensure your examples stay up-to-date with your code.

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Write unit tests using the `#[test]` attribute
2. Organize tests with modules and integration tests
3. Use assertions and test utilities effectively
4. Write documentation comments and doctests
5. Apply testing best practices and patterns

---

## 🧠 Key Concepts

### Testing in Rust

**Test Types**:
- **Unit Tests**: Test individual functions/modules
- **Integration Tests**: Test public API from external perspective
- **Doc Tests**: Test code examples in documentation

### Test Organization

- **#[cfg(test)]**: Conditional compilation for tests
- **tests/ Directory**: Integration tests location
- **Assertions**: `assert!`, `assert_eq!`, `assert_ne!`
- **Test Attributes**: `#[test]`, `#[should_panic]`, `#[ignore]`

---

## 🔬 Live Code Exploration

### Basic Unit Testing

In [ ]:
// Basic unit testing examples

// Function to test
fn add(a: i32, b: i32) -> i32 {
    a + b
}

fn divide(a: f64, b: f64) -> Result<f64, String> {
    if b == 0.0 {
        Err("Division by zero".to_string())
    } else {
        Ok(a / b)
    }
}

fn is_even(n: i32) -> bool {
    n % 2 == 0
}

fn factorial(n: u32) -> u32 {
    match n {
        0 | 1 => 1,
        _ => n * factorial(n - 1),
    }
}

// Test module
#[cfg(test)]
mod tests {
    use super::*; // Import functions from parent module
    
    #[test]
    fn test_add() {
        assert_eq!(add(2, 3), 5);
        assert_eq!(add(-1, 1), 0);
        assert_eq!(add(0, 0), 0);
    }
    
    #[test]
    fn test_divide_success() {
        let result = divide(10.0, 2.0);
        assert!(result.is_ok());
        assert_eq!(result.unwrap(), 5.0);
    }
    
    #[test]
    fn test_divide_by_zero() {
        let result = divide(10.0, 0.0);
        assert!(result.is_err());
        assert_eq!(result.unwrap_err(), "Division by zero");
    }
    
    #[test]
    fn test_is_even() {
        assert!(is_even(2));
        assert!(is_even(0));
        assert!(is_even(-4));
        assert!(!is_even(1));
        assert!(!is_even(-3));
    }
    
    #[test]
    fn test_factorial() {
        assert_eq!(factorial(0), 1);
        assert_eq!(factorial(1), 1);
        assert_eq!(factorial(5), 120);
        assert_eq!(factorial(3), 6);
    }
    
    #[test]
    #[should_panic]
    fn test_factorial_overflow() {
        // This should panic due to overflow
        factorial(50);
    }
    
    #[test]
    #[ignore]
    fn expensive_test() {
        // This test is ignored by default
        // Run with: cargo test -- --ignored
        assert_eq!(factorial(10), 3628800);
    }
}

// Demonstrate running tests (in a real project, you'd use `cargo test`)
fn demonstrate_testing() {
    println!("=== Testing Demonstration ===");
    println!("In a real Rust project, you would run: cargo test");
    println!("This would execute all functions marked with #[test]");
    
    // Manual test execution for demonstration
    println!("\nManual test results:");
    println!("add(2, 3) = {} (expected: 5)", add(2, 3));
    println!("divide(10.0, 2.0) = {:?} (expected: Ok(5.0))", divide(10.0, 2.0));
    println!("divide(10.0, 0.0) = {:?} (expected: Err)", divide(10.0, 0.0));
    println!("is_even(4) = {} (expected: true)", is_even(4));
    println!("factorial(5) = {} (expected: 120)", factorial(5));
}

demonstrate_testing();

In [ ]:
// Advanced testing patterns and utilities

#[derive(Debug, PartialEq)]
struct BankAccount {
    balance: f64,
    account_number: String,
}

impl BankAccount {
    fn new(account_number: String, initial_balance: f64) -> Result<Self, String> {
        if initial_balance < 0.0 {
            return Err("Initial balance cannot be negative".to_string());
        }
        
        Ok(BankAccount {
            balance: initial_balance,
            account_number,
        })
    }
    
    fn deposit(&mut self, amount: f64) -> Result<(), String> {
        if amount <= 0.0 {
            return Err("Deposit amount must be positive".to_string());
        }
        
        self.balance += amount;
        Ok(())
    }
    
    fn withdraw(&mut self, amount: f64) -> Result<(), String> {
        if amount <= 0.0 {
            return Err("Withdrawal amount must be positive".to_string());
        }
        
        if amount > self.balance {
            return Err("Insufficient funds".to_string());
        }
        
        self.balance -= amount;
        Ok(())
    }
    
    fn balance(&self) -> f64 {
        self.balance
    }
    
    fn transfer(&mut self, other: &mut BankAccount, amount: f64) -> Result<(), String> {
        self.withdraw(amount)?;
        match other.deposit(amount) {
            Ok(()) => Ok(()),
            Err(e) => {
                // Rollback the withdrawal
                self.balance += amount;
                Err(e)
            }
        }
    }
}

#[cfg(test)]
mod bank_tests {
    use super::*;
    
    // Helper function for creating test accounts
    fn create_test_account(balance: f64) -> BankAccount {
        BankAccount::new("TEST123".to_string(), balance).unwrap()
    }
    
    #[test]
    fn test_account_creation() {
        let account = BankAccount::new("ACC001".to_string(), 100.0);
        assert!(account.is_ok());
        
        let account = account.unwrap();
        assert_eq!(account.balance(), 100.0);
        assert_eq!(account.account_number, "ACC001");
    }
    
    #[test]
    fn test_negative_initial_balance() {
        let account = BankAccount::new("ACC002".to_string(), -50.0);
        assert!(account.is_err());
        assert_eq!(account.unwrap_err(), "Initial balance cannot be negative");
    }
    
    #[test]
    fn test_deposit() {
        let mut account = create_test_account(100.0);
        
        let result = account.deposit(50.0);
        assert!(result.is_ok());
        assert_eq!(account.balance(), 150.0);
    }
    
    #[test]
    fn test_invalid_deposit() {
        let mut account = create_test_account(100.0);
        
        let result = account.deposit(-10.0);
        assert!(result.is_err());
        assert_eq!(account.balance(), 100.0); // Balance unchanged
    }
    
    #[test]
    fn test_withdrawal() {
        let mut account = create_test_account(100.0);
        
        let result = account.withdraw(30.0);
        assert!(result.is_ok());
        assert_eq!(account.balance(), 70.0);
    }
    
    #[test]
    fn test_insufficient_funds() {
        let mut account = create_test_account(50.0);
        
        let result = account.withdraw(100.0);
        assert!(result.is_err());
        assert_eq!(result.unwrap_err(), "Insufficient funds");
        assert_eq!(account.balance(), 50.0); // Balance unchanged
    }
    
    #[test]
    fn test_transfer() {
        let mut account1 = create_test_account(100.0);
        let mut account2 = create_test_account(50.0);
        
        let result = account1.transfer(&mut account2, 30.0);
        assert!(result.is_ok());
        assert_eq!(account1.balance(), 70.0);
        assert_eq!(account2.balance(), 80.0);
    }
    
    #[test]
    fn test_transfer_insufficient_funds() {
        let mut account1 = create_test_account(20.0);
        let mut account2 = create_test_account(50.0);
        
        let result = account1.transfer(&mut account2, 30.0);
        assert!(result.is_err());
        // Both balances should be unchanged
        assert_eq!(account1.balance(), 20.0);
        assert_eq!(account2.balance(), 50.0);
    }
    
    // Parameterized test using a macro
    macro_rules! test_deposit_amounts {
        ($($name:ident: $value:expr,)*) => {
            $(
                #[test]
                fn $name() {
                    let (initial, deposit, expected) = $value;
                    let mut account = create_test_account(initial);
                    account.deposit(deposit).unwrap();
                    assert_eq!(account.balance(), expected);
                }
            )*
        }
    }
    
    test_deposit_amounts! {
        test_deposit_small: (100.0, 10.0, 110.0),
        test_deposit_large: (100.0, 1000.0, 1100.0),
        test_deposit_decimal: (100.0, 25.50, 125.50),
    }
}

fn advanced_testing_demo() {
    println!("\n=== Advanced Testing Patterns ===");
    println!("Advanced testing includes:");
    println!("- Helper functions for test setup");
    println!("- Testing error conditions");
    println!("- Parameterized tests with macros");
    println!("- State verification after operations");
    
    // Demonstrate the bank account functionality
    let mut account1 = BankAccount::new("ACC001".to_string(), 1000.0).unwrap();
    let mut account2 = BankAccount::new("ACC002".to_string(), 500.0).unwrap();
    
    println!("\nDemonstration:");
    println!("Account 1 balance: ${:.2}", account1.balance());
    println!("Account 2 balance: ${:.2}", account2.balance());
    
    account1.transfer(&mut account2, 200.0).unwrap();
    println!("\nAfter transferring $200 from Account 1 to Account 2:");
    println!("Account 1 balance: ${:.2}", account1.balance());
    println!("Account 2 balance: ${:.2}", account2.balance());
}

advanced_testing_demo();